In [7]:
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
sys.path.append('./vrep_lib')
from vrep_lib import vrep
import pandas

from Simulator import *

In [ ]:
from version import VERSION


In [8]:
s = Simulator()

Connected to remote API server
left:Error code: 0
robot:Error code: 0
right:Error code: 0
cam:Error code: 0
cam:Error code: 0


In [13]:
vrep.simxStartSimulation(s.clientID, vrep.simx_opmode_oneshot)


0

In [12]:
vrep.simxPauseSimulation(s.clientID, vrep.simx_opmode_oneshot)

1

In [14]:
vrep.simxStopSimulation(s.clientID, vrep.simx_opmode_oneshot)

0

In [2]:
path = pandas.read_csv('./path1.csv',',')

In [11]:
path_array = path.as_matrix()
print(path_array.shape)
path_array = path_array[:,:6]
print(path_array.shape)
path_array

(966, 16)
(966, 6)


array([[  8.715327, -25.398006,   0.138793,  -0.      ,   0.      ,
         11.47834 ],
       [  8.724535, -25.394106,   0.138793,  -0.      ,   0.      ,
         22.956682],
       [  8.732783, -25.388451,   0.138793,  -0.      ,   0.      ,
         34.435024],
       ..., 
       [  7.836516, -17.494106,   0.138793,   0.      ,   0.      ,
        -22.956682],
       [  7.845724, -17.498007,   0.138793,   0.      ,   0.      ,
        -11.47834 ],
       [  7.855524, -17.499996,   0.138793,  -0.      ,   0.      ,  -0.      ]])

In [12]:
path_array[10,:]

array([  8.767263, -25.317076,   0.138793,  -0.      ,   0.      ,  68.3521  ])

In [14]:
print(np.linalg.norm([1,1,1]))

1.73205080757


In [3]:
from approxeng.input.controllers import print_devices

print_devices()

{ 'axes': { 'ABS_HAT0X': {'code': 16, 'flat': 0, 'fuzz': 0, 'max': 1, 'min': -1, 'res': 0},
            'ABS_HAT0Y': {'code': 17, 'flat': 0, 'fuzz': 0, 'max': 1, 'min': -1, 'res': 0},
            'ABS_RX': {'code': 3, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_RY': {'code': 4, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_RZ': {'code': 5, 'flat': 0, 'fuzz': 0, 'max': 1023, 'min': 0, 'res': 0},
            'ABS_X': {'code': 0, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_Y': {'code': 1, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_Z': {'code': 2, 'flat': 0, 'fuzz': 0, 'max': 1023, 'min': 0, 'res': 0}},
  'bus': 3,
  'buttons': { 304: ['BTN_A', 'BTN_GAMEPAD', 'BTN_SOUTH'],
               305: ['BTN_B', 'BTN_EAST'],
               307: ['BTN_NORTH', 'BTN_X'],
               308: ['BTN_WEST', 'BTN_Y'],
               310: 'BTN_TL',
           

In [4]:
print_devices()

{ 'axes': { 'ABS_HAT0X': {'code': 16, 'flat': 0, 'fuzz': 0, 'max': 1, 'min': -1, 'res': 0},
            'ABS_HAT0Y': {'code': 17, 'flat': 0, 'fuzz': 0, 'max': 1, 'min': -1, 'res': 0},
            'ABS_RX': {'code': 3, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_RY': {'code': 4, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_RZ': {'code': 5, 'flat': 0, 'fuzz': 0, 'max': 1023, 'min': 0, 'res': 0},
            'ABS_X': {'code': 0, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_Y': {'code': 1, 'flat': 128, 'fuzz': 16, 'max': 32767, 'min': -32768, 'res': 0},
            'ABS_Z': {'code': 2, 'flat': 0, 'fuzz': 0, 'max': 1023, 'min': 0, 'res': 0}},
  'bus': 3,
  'buttons': { 304: ['BTN_A', 'BTN_GAMEPAD', 'BTN_SOUTH'],
               305: ['BTN_B', 'BTN_EAST'],
               307: ['BTN_NORTH', 'BTN_X'],
               308: ['BTN_WEST', 'BTN_Y'],
               310: 'BTN_TL',
           

In [5]:
from approxeng.input.selectbinder import ControllerResource
from time import sleep
import curses
import pprint

def main(screen):
    curses.curs_set(False)
    curses.init_pair(1, curses.COLOR_RED, curses.COLOR_BLACK)
    curses.init_pair(2, curses.COLOR_GREEN, curses.COLOR_BLACK)
    curses.init_pair(3, curses.COLOR_YELLOW, curses.COLOR_BLACK)
    curses.start_color()
    last_presses = None

    def red(s):
        screen.addstr(s, curses.color_pair(1))

    def green(s):
        screen.addstr(s, curses.color_pair(2))

    def yellow(s):
        screen.addstr(s, curses.color_pair(3))

    # Loop forever
    while True:
        try:
            with ControllerResource() as joystick:
                while joystick.connected:
                    # Check for presses since the last time we checked
                    joystick.check_presses()

                    screen.clear()

                    if joystick.has_presses:
                        last_presses = joystick.presses

                    # Print most recent presses set
                    screen.addstr(0, 0, 'last presses:')
                    if last_presses is not None:
                        for button_name in last_presses:
                            green(' {}'.format(button_name))

                    # Print axis values
                    screen.addstr(1, 0, 'axes:')
                    for axis_name in joystick.axes.names:
                        screen.addstr(' {}='.format(axis_name))
                        axis_value = joystick[axis_name]
                        if axis_value > 0:
                            green('{:.2f}'.format(axis_value))
                        elif axis_value < 0:
                            red('{:.2f}'.format(axis_value))
                        else:
                            yellow('{:.2f}'.format(axis_value))

                    # Print button hold times
                    screen.addstr(2, 0, 'hold times:')
                    for button_name in joystick.buttons.names:
                        hold_time = joystick[button_name]
                        if hold_time is not None:
                            screen.addstr(' {}='.format(button_name))
                            green('{:.1f}'.format(hold_time))

                    # Print some details of the controller
                    screen.addstr(3, 0, 'controller class: {}'.format(type(joystick).__name__))
                    screen.addstr(5, 0, pprint.pformat(joystick.controls, indent=2))

                    screen.refresh()
        except IOError:
            screen.clear()
            screen.addstr(0, 0, 'Waiting for controller')
            screen.refresh()
            sleep(1.0)


curses.wrapper(main)

error: curs_set() returned ERR